<a href="https://colab.research.google.com/github/ashijainn/Amazon_Clone/blob/main/Traffic_signal_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

Install the Cirq and qsimcirq packages:

In [ ]:
try:
    import cirq
except ImportError:
    !pip install cirq --quiet
    import cirq

try:
    import qsimcirq
except ImportError:
    !pip install qsimcirq --quiet
    import qsimcirq

Simulating Cirq circuits with qsim is easy: just define the circuit as you normally would, then create a `QSimSimulator` to perform the simulation. This object implements Cirq's [simulator.py](https://github.com/quantumlib/Cirq/blob/master/cirq-core/cirq/sim/simulator.py) interfaces, so you can drop it in anywhere the basic Cirq simulator is used.

## Full state-vector simulation

qsim is optimized for computing the final state vector of a circuit. Try it by running the example below.

In [ ]:
# Define qubits and a short circuit.
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(cirq.H(q0), cirq.CX(q0, q1))
print("Circuit:")
print(circuit)
print()

# Simulate the circuit with Cirq and return the full state vector.
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.simulate(circuit)
print(cirq_results)
print()

# Simulate the circuit with qsim and return the full state vector.
print('qsim results:')
qsim_simulator = qsimcirq.QSimSimulator()
qsim_results = qsim_simulator.simulate(circuit)
print(qsim_results)

To sample from this state, you can invoke Cirq's `sample_state_vector` method:

In [ ]:
samples = cirq.sample_state_vector(
    qsim_results.state_vector(), indices=[0, 1], repetitions=10)
print(samples)

## Measurement sampling

qsim also supports sampling from user-defined measurement gates.

> *Note*: Since qsim and Cirq use different random number generators, identical runs on both simulators may give different results, even if they use the same seed.

In [ ]:
# Define a circuit with measurements.
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(
    cirq.H(q0), cirq.X(q1), cirq.CX(q0, q1),
    cirq.measure(q0, key='qubit_0'),
    cirq.measure(q1, key='qubit_1'),
)
print("Circuit:")
print(circuit)
print()

# Simulate the circuit with Cirq and return just the measurement values.
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.run(circuit, repetitions=5)
print(cirq_results)
print()

# Simulate the circuit with qsim and return just the measurement values.
print('qsim results:')
qsim_simulator = qsimcirq.QSimSimulator()
qsim_results = qsim_simulator.run(circuit, repetitions=5)
print(qsim_results)

The warning above highlights an important distinction between the `simulate` and `run` methods:

* `simulate` only executes the circuit once.
  -  Sampling from the resulting state is fast, but if there are intermediate measurements the final state vector depends on the results of those measurements.
* `run` will execute the circuit once for each repetition requested.
  -  As a result, sampling is much slower, but intermediate measurements are re-sampled for each repetition. If there are no intermediate measurements, `run` redirects to `simulate` for faster execution.

The warning goes away if intermediate measurements are present:

In [ ]:
# Define a circuit with intermediate measurements.
q0 = cirq.LineQubit(0)
circuit = cirq.Circuit(
    cirq.X(q0)**0.5, cirq.measure(q0, key='m0'),
    cirq.X(q0)**0.5, cirq.measure(q0, key='m1'),
    cirq.X(q0)**0.5, cirq.measure(q0, key='m2'),
)
print("Circuit:")
print(circuit)
print()

# Simulate the circuit with qsim and return just the measurement values.
print('qsim results:')
qsim_simulator = qsimcirq.QSimSimulator()
qsim_results = qsim_simulator.run(circuit, repetitions=5)
print(qsim_results)

## Amplitude evaluation

qsim can also calculate amplitudes for specific output bitstrings.

In [ ]:
# Define a simple circuit.
q0, q1 = cirq.LineQubit.range(2)
circuit = cirq.Circuit(cirq.H(q0), cirq.CX(q0, q1))
print("Circuit:")
print(circuit)
print()

# Simulate the circuit with qsim and return the amplitudes for |00) and |01).
print('Cirq results:')
cirq_simulator = cirq.Simulator()
cirq_results = cirq_simulator.compute_amplitudes(
    circuit, bitstrings=[0b00, 0b01])
print(cirq_results)
print()

# Simulate the circuit with qsim and return the amplitudes for |00) and |01).
print('qsim results:')
qsim_simulator = qsimcirq.QSimSimulator()
qsim_results = qsim_simulator.compute_amplitudes(
    circuit, bitstrings=[0b00, 0b01])
print(qsim_results)

## Performance benchmark

The code below generates a depth-16 circuit on a 4x5 qubit grid, then runs it against the basic Cirq simulator. For a circuit of this size, the difference in runtime can be significant - try it out!

In [ ]:
import time

# Get a rectangular grid of qubits.
qubits = cirq.GridQubit.rect(4, 5)

# Generates a random circuit on the provided qubits.
circuit = cirq.experiments.random_rotations_between_grid_interaction_layers_circuit(
    qubits=qubits, depth=16)

# Simulate the circuit with Cirq and print the runtime.
cirq_simulator = cirq.Simulator()
cirq_start = time.time()
cirq_results = cirq_simulator.simulate(circuit)
cirq_elapsed = time.time() - cirq_start
print(f'Cirq runtime: {cirq_elapsed} seconds.')
print()

# Simulate the circuit with qsim and print the runtime.
qsim_simulator = qsimcirq.QSimSimulator()
qsim_start = time.time()
qsim_results = qsim_simulator.simulate(circuit)
qsim_elapsed = time.time() - qsim_start
print(f'qsim runtime: {qsim_elapsed} seconds.')

qsim performance can be tuned further by passing options to the simulator constructor. These options use the same format as the qsim_base binary - a full description can be found in the qsim [usage doc](https://github.com/quantumlib/qsim/blob/master/docs/usage.md). The example below demonstrates enabling multithreading in qsim; for best performance, use the same number of threads as the number of cores (or virtual cores) on your machine.

In [ ]:
# Use eight threads to parallelize simulation.
options = {'t': 8}

qsim_simulator = qsimcirq.QSimSimulator(options)
qsim_start = time.time()
qsim_results = qsim_simulator.simulate(circuit)
qsim_elapsed = time.time() - qsim_start
print(f'qsim runtime: {qsim_elapsed} seconds.')

Another option is to adjust the maximum number of qubits over which to fuse gates. Increasing this value (as demonstrated below) increases arithmetic intensity, which may improve performance with the right environment settings.

In [ ]:
# Increase maximum fused gate size to three qubits.
options = {'f': 3}

qsim_simulator = qsimcirq.QSimSimulator(options)
qsim_start = time.time()
qsim_results = qsim_simulator.simulate(circuit)
qsim_elapsed = time.time() - qsim_start
print(f'qsim runtime: {qsim_elapsed} seconds.')

## Advanced applications: Distributed execution

qsimh (qsim-hybrid) is a second library in the qsim repository that takes a slightly different approach to circuit simulation. When simulating a quantum circuit, it's possible to simplify the execution by decomposing a subset of two-qubit gates into pairs of one-qubit gates with shared indices. This operation is called "slicing" (or "cutting") the gates.

qsimh takes advantage of the "slicing" operation by selecting a set of gates to "slice" and assigning each possible value of the shared indices across a set of executors running in parallel. By adding up the results afterwards, the total state can be recovered.

In [ ]:
# Pick a pair of qubits.
q0 = cirq.GridQubit(0, 0)
q1 = cirq.GridQubit(0, 1)

# Create a circuit that entangles the pair.
circuit = cirq.Circuit(
    cirq.H(q0), cirq.CX(q0, q1), cirq.X(q1)
)
print("Circuit:")
print(circuit)

In order to let qsimh know how we want to split up the circuit, we need to pass it some additional options. More detail on these can be found in the qsim [usage doc](https://github.com/quantumlib/qsim/blob/master/docs/usage.md), but the fundamentals are explained below.

In [ ]:
options = {}

# 'k' indicates the qubits on one side of the cut.
# We'll use qubit 0 for this.
options['k'] = [0]

# 'p' and 'r' control when values are assigned to cut indices.
# There are some intricacies in choosing values for these options,
# but for now we'll set p=1 and r=0.
# This allows us to pre-assign the value of the CX indices
# and distribute its execution to multiple jobs.
options['p'] = 1
options['r'] = 0

# 'w' indicates the value pre-assigned to the cut.
# This should change for each execution.
options['w'] = 0

# Create the qsimh simulator with those options.
qsimh_simulator = qsimcirq.QSimhSimulator(options)
results_0 = qsimh_simulator.compute_amplitudes(
    circuit, bitstrings=[0b00, 0b01, 0b10, 0b11])
print(results_0)

Now to run the other side of the cut...

In [ ]:
options['w'] = 1

qsimh_simulator = qsimcirq.QSimhSimulator(options)
results_1 = qsimh_simulator.compute_amplitudes(
    circuit, bitstrings=[0b00, 0b01, 0b10, 0b11])
print(results_1)

...and add the two together. The results of a normal qsim simulation are shown for comparison.

In [ ]:
results = [r0 + r1 for r0, r1 in zip(results_0, results_1)]
print("qsimh results:")
print(results)

qsim_simulator = qsimcirq.QSimSimulator()
qsim_simulator.compute_amplitudes(circuit, bitstrings=[0b00, 0b01, 0b10, 0b11])
print("qsim results:")
print(results)

The key point to note here is that `results_0` and `results_1` are completely independent - they can be run in parallel on two separate machines, with no communication between the two. Getting the full result requires `2^p` executions, but each individual result is much cheaper to calculate than trying to do the whole circuit at once.

In [ ]:
import heapq

# Define the road network as a graph (nodes are intersections, edges are roads with weights)
roads = {
    'A': {'B': 10, 'C': 15},
    'B': {'D': 20},
    'C': {'D': 10},
    'D': {}
}


Graph Representation: The road network is represented as a dictionary of dictionaries. Each key is a node (e.g., 'A', 'B'), and the value is another dictionary where the keys are connected nodes and the values are weights (e.g., travel time or distance). In this case, road weights are represented as travel times between intersections.

##Dijkstra's Algorithm

In [ ]:
def dijkstra(graph, start, end):
    # Priority queue to store the minimum distance from start to end
    queue = [(0, start, [])]
    seen = set()
    min_dist = {start: 0}

    while queue:
        (cost, node, path) = heapq.heappop(queue)

        if node in seen:
            continue

        path = path + [node]
        seen.add(node)

        if node == end:
            return cost, path

        for next_node, weight in graph[node].items():
            if next_node in seen:
                continue
            prev_cost = min_dist.get(next_node, float('inf'))
            next_cost = cost + weight
            if next_cost < prev_cost:
                min_dist[next_node] = next_cost
                heapq.heappush(queue, (next_cost, next_node, path))

    return float('inf'), []


Priority Queue: The algorithm uses a priority queue (heapq) to explore nodes. The queue stores the total cost to reach a node, the node itself, and the path taken so far. The priority is based on the total cost (shortest path).

Tracking Seen Nodes: The algorithm keeps track of nodes that have been visited (seen set) to avoid re-exploring them.

Cost Calculation: For each node, Dijkstra's Algorithm checks all neighboring nodes. If a neighboring node hasn't been visited, it calculates the cost to reach that neighbor, updates the minimum cost if it's smaller than the previously recorded cost, and adds it to the queue.

Path Finding: The algorithm stops once it reaches the destination (end). It returns the total travel cost and the path taken.

In [ ]:
cost, path = dijkstra(roads, 'A', 'D')
print(f"Shortest path: {path} with cost {cost}")


Shortest Path Output: The result will show the shortest path from node 'A' to 'D' with the corresponding total cost.

##Quantum-Inspired Genetic Algorithms (QIGAs) for Vehicle Rerouting

In [ ]:
population = [
    ['A', 'B', 'D'],  # Route 1
    ['A', 'C', 'D'],  # Route 2
    ['A', 'B', 'C', 'D']  # Route 3
]

def fitness(route, graph):
    total_cost = 0
    for i in range(len(route) - 1):
        current_node = route[i]
        next_node = route[i+1]

        # Check if the edge exists in the graph before accessing it
        if next_node in graph.get(current_node, {}):
            total_cost += graph[current_node][next_node]
        else:
            # If the edge doesn't exist, assign a very high cost (infinity)
            return float('inf')  # Invalid route

    return 1 / total_cost  # Higher fitness for shorter travel time


Population: The population is a set of possible routes (chromosomes). Each route is a list of nodes representing a potential path from the vehicle's current location to its destination.

Fitness Function: The fitness of each route is determined by the total cost of traveling that route. If an edge in the route doesn't exist (i.e., no road between two nodes), the route is marked as invalid with a very high cost (float('inf')). The fitness score is the inverse of the total travel time, so shorter routes have higher fitness.

In [ ]:
def crossover(route1, route2):
    cut_point = random.randint(1, min(len(route1), len(route2)) - 2)
    return route1[:cut_point] + route2[cut_point:]

def mutate(route):
    if len(route) > 2:  # Ensure there are at least two nodes to swap
        i, j = random.sample(range(1, len(route) - 1), 2)  # Avoid start and end nodes
        route[i], route[j] = route[j], route[i]
    return route


Crossover: Combines two routes (parents) to produce a new route (child). The crossover operation cuts each route at a random point and combines parts of the two parent routes to form a new path. This allows exploration of new paths that weren’t in the original population.

Mutation: Randomly swaps two nodes in the route (except for the start and end nodes) to introduce variability. This simulates changes in the vehicle’s route and helps the algorithm explore new possibilities, avoiding local optima.

##Quantum-Inspired Genetic Algorithm

In [ ]:
def quantum_genetic_algorithm(population, graph, generations=10):
    for generation in range(generations):
        population.sort(key=lambda route: fitness(route, graph), reverse=True)
        # Select the top routes
        new_population = population[:2]
        # Apply crossover and mutation
        for _ in range(len(population) - 2):
            parent1, parent2 = random.sample(population, 2)
            child = crossover(parent1, parent2)
            if random.random() < 0.1:  # Mutation chance
                child = mutate(child)
            new_population.append(child)
        population = new_population
        print(f"Generation {generation+1}: Best route {population[0]} with fitness {fitness(population[0], graph)}")

    return population[0]


Generations: The algorithm evolves the population over a number of generations (iterations). In each generation:

The population is sorted by fitness, and the best routes are kept.
New routes are created by applying crossover and mutation.
The population evolves until an optimal or near-optimal solution is found.
Random Selection: Parents for crossover are selected randomly from the population, and there’s a small chance (e.g., 10%) that mutation occurs to introduce new routes.

In [ ]:
best_route = quantum_genetic_algorithm(population, roads)
print(f"Best rerouted path: {best_route}")


Best Route Output: The algorithm prints the best route for each generation and eventually returns the most optimal route after evolving the population.

In [ ]:
def quantum_annealing_traffic_light_optimization(intersections):
    # Each traffic light signal's green/red state is a binary variable in a QUBO problem
    for intersection in intersections:
        # Define the QUBO variables and constraints for each intersection
        green_duration = anneal(intersection)
        print(f"Optimized green duration for {intersection}: {green_duration}")

# Example use (pseudo): apply quantum annealing to optimize traffic light timings
quantum_annealing_traffic_light_optimization(['A', 'B', 'C', 'D'])
